In [1]:
!git clone https://github.com/ajay-sainy/Wav2Lip-GFPGAN.git
basePath = "/content/Wav2Lip-GFPGAN"
%cd {basePath}

Cloning into 'Wav2Lip-GFPGAN'...
remote: Enumerating objects: 195, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 195 (delta 18), reused 9 (delta 9), pack-reused 160 (from 1)
Receiving objects: 100% (195/195), 29.94 MiB | 26.63 MiB/s, done.
Resolving deltas: 100% (39/39), done.
/content/Wav2Lip-GFPGAN


In [2]:
wav2lipFolderName = 'Wav2Lip-master'
gfpganFolderName = 'GFPGAN-master'
wav2lipPath = basePath + '/' + wav2lipFolderName
gfpganPath = basePath + '/' + gfpganFolderName

!wget 'https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth' -O {wav2lipPath}'/face_detection/detection/sfd/s3fd.pth'
!gdown https://drive.google.com/uc?id=1fQtBSYEyuai9MjBOF8j7zZ4oQ9W2N64q --output {wav2lipPath}'/checkpoints/'

--2025-04-07 13:27:13--  https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth
Resolving www.adrianbulat.com (www.adrianbulat.com)... 45.136.29.207
Connecting to www.adrianbulat.com (www.adrianbulat.com)|45.136.29.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89843225 (86M) [application/octet-stream]
Saving to: ‘/content/Wav2Lip-GFPGAN/Wav2Lip-master/face_detection/detection/sfd/s3fd.pth’

/content/Wav2Lip-GF 100%[===================>]  85.68M   183MB/s    in 0.5s    

2025-04-07 13:27:14 (183 MB/s) - ‘/content/Wav2Lip-GFPGAN/Wav2Lip-master/face_detection/detection/sfd/s3fd.pth’ saved [89843225/89843225]

Downloading...
From (original): https://drive.google.com/uc?id=1fQtBSYEyuai9MjBOF8j7zZ4oQ9W2N64q
From (redirected): https://drive.google.com/uc?id=1fQtBSYEyuai9MjBOF8j7zZ4oQ9W2N64q&confirm=t&uuid=ba76ff21-ade9-4e2a-9db9-86c6676b4de6
To: /content/Wav2Lip-GFPGAN/Wav2Lip-master/checkpoints/wav2lip.pth
100% 436M/436M [00:09<00:00, 44.1MB/s]


In [3]:
!pip install -r requirements.txt

ERROR: Invalid requirement: 'torchvision>=>=0.8.2': Expected end or semicolon (after name and no valid version specifier)
    torchvision>=>=0.8.2
               ^ (from line 6 of requirements.txt)


In [ ]:
# Install required libraries
!pip install flask pyngrok gdown

# Clone the repository
!git clone https://github.com/ajay-sainy/Wav2Lip-GFPGAN.git

# Set the base path
basePath = "/content/Wav2Lip-GFPGAN"
%cd {basePath}

# Define folder names
wav2lipFolderName = 'Wav2Lip-master'
gfpganFolderName = 'GFPGAN-master'

# Define paths
wav2lipPath = basePath + '/' + wav2lipFolderName
gfpganPath = basePath + '/' + gfpganFolderName

# Download the pretrained model for face detection
!wget 'https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth' -O {wav2lipPath}/face_detection/detection/sfd/s3fd.pth

# Download the Wav2Lip pretrained model
!gdown https://drive.google.com/uc?id=1fQtBSYEyuai9MjBOF8j7zZ4oQ9W2N64q --output {wav2lipPath}/checkpoints/wav2lip.pth

# Install dependencies
!pip install -r requirements.txt

# Import necessary libraries
from flask import Flask, request, jsonify, send_file
import os
from werkzeug.utils import secure_filename
from pyngrok import ngrok

# Initialize Flask app
app = Flask(__name__)

# Set up paths
inputPath = basePath + '/inputs'
outputPath = basePath + '/outputs'

# Create input and output directories if they don't exist
if not os.path.exists(inputPath):
    os.makedirs(inputPath)
if not os.path.exists(outputPath):
    os.makedirs(outputPath)

# Define allowed file extensions
ALLOWED_EXTENSIONS = {'mp4', 'wav'}

# Function to check if a file has an allowed extension
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

# Flask route to handle file upload and processing
@app.route('/process', methods=['POST'])
def process_files():
    # Check if files are present in the request
    if 'video' not in request.files or 'audio' not in request.files:
        return jsonify({"error": "Both video and audio files are required"}), 400

    video_file = request.files['video']
    audio_file = request.files['audio']

    # Check if files have valid extensions
    if not allowed_file(video_file.filename) or not allowed_file(audio_file.filename):
        return jsonify({"error": "Invalid file type. Only .mp4 and .wav files are allowed"}), 400

    # Save the uploaded files
    video_path = os.path.join(inputPath, secure_filename(video_file.filename))
    audio_path = os.path.join(inputPath, secure_filename(audio_file.filename))
    video_file.save(video_path)
    audio_file.save(audio_path)

    # Define the output path for the lip-synced video
    lipSyncedOutputPath = os.path.join(outputPath, 'result.mp4')

    # Run the Wav2Lip inference
    !cd {wav2lipPath} && python inference.py \
        --checkpoint_path checkpoints/wav2lip.pth \
        --face {video_path} \
        --audio {audio_path} \
        --outfile {lipSyncedOutputPath}

    # Return the output file as a downloadable link
    return send_file(lipSyncedOutputPath, as_attachment=True)

# Start the Flask server
if __name__ == '__main__':
    # Set your ngrok token here
    ngrok.set_auth_token("2rrH36aY2OjghIDbMHyH1e9TO8u_674w937bU9yjRn69FhMVX")

    # Expose the Flask app via ngrok
    public_url = ngrok.connect(5000).public_url
    print(f" * Running on {public_url}")

    # Run the Flask app
    app.run(host='0.0.0.0', port=5000)

Cloning into 'Wav2Lip-GFPGAN'...
remote: Enumerating objects: 195, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 195 (delta 18), reused 9 (delta 9), pack-reused 160 (from 1)
Receiving objects: 100% (195/195), 29.94 MiB | 16.28 MiB/s, done.
Resolving deltas: 100% (40/40), done.
/content/Wav2Lip-GFPGAN
--2025-02-17 19:16:30--  https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth
Resolving www.adrianbulat.com (www.adrianbulat.com)... 45.136.29.207
Connecting to www.adrianbulat.com (www.adrianbulat.com)|45.136.29.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89843225 (86M) [application/octet-stream]
Saving to: ‘/content/Wav2Lip-GFPGAN/Wav2Lip-master/face_detection/detection/sfd/s3fd.pth’

/content/Wav2Lip-GF 100%[===================>]  85.68M  18.7MB/s    in 5.7s    

2025-02-17 19:16:37 (15.0 MB/s) - ‘/content/Wav2Lip-GFPGAN/Wav2Lip-master/face_detection/detection/sfd/s3fd.

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


Using cuda for inference.
Reading video frames...
Number of frames available for inference: 169
(80, 1266)
Length of mel chunks: 392
  0% 0/4 [00:00<?, ?it/s]/content/Wav2Lip-GFPGAN/Wav2Lip-master/face_detection/detection/sfd/sfd_detector.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have 

INFO:werkzeug:127.0.0.1 - - [17/Feb/2025 19:19:19] "POST /process HTTP/1.1" 200 -


In [ ]:
# Install required libraries
!pip install flask pyngrok gdown

# Clone the repository
!git clone https://github.com/ajay-sainy/Wav2Lip-GFPGAN.git

# Set the base path
basePath = "/content/Wav2Lip-GFPGAN"
%cd {basePath}

# Define folder names
wav2lipFolderName = 'Wav2Lip-master'
gfpganFolderName = 'GFPGAN-master'

# Define paths
wav2lipPath = basePath + '/' + wav2lipFolderName
gfpganPath = basePath + '/' + gfpganFolderName

# Download the pretrained model for face detection
!wget 'https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth' -O {wav2lipPath}/face_detection/detection/sfd/s3fd.pth

# Download the Wav2Lip pretrained model
!gdown https://drive.google.com/uc?id=1fQtBSYEyuai9MjBOF8j7zZ4oQ9W2N64q --output {wav2lipPath}/checkpoints/wav2lip.pth

# Install dependencies
!pip install -r requirements.txt

# Import necessary libraries
from flask import Flask, request, jsonify, send_file
import os
from werkzeug.utils import secure_filename
from pyngrok import ngrok
from concurrent.futures import ProcessPoolExecutor
import subprocess
import uuid
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize Flask app
app = Flask(__name__)

# Set up paths
inputPath = basePath + '/inputs'
outputPath = basePath + '/outputs'

# Create input and output directories if they don’t exist
os.makedirs(inputPath, exist_ok=True)
os.makedirs(outputPath, exist_ok=True)

# Define allowed file extensions
ALLOWED_EXTENSIONS = {'mp4', 'wav'}

# Function to check if a file has an allowed extension
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

# Initialize process pool with max_workers=1 to limit concurrent inference processes
executor = ProcessPoolExecutor(max_workers=1)

# Function to run Wav2Lip inference
def run_inference(video_path, audio_path, output_path):
    cmd = [
        'python', 'inference.py',
        '--checkpoint_path', 'checkpoints/wav2lip.pth',
        '--face', video_path,
        '--audio', audio_path,
        '--outfile', output_path
    ]
    subprocess.run(cmd, cwd=wav2lipPath, check=True)

# Flask route to handle file upload and processing
@app.route('/process', methods=['POST'])
def process_files():
    logging.info("Received a new request")

    # Check if files are present in the request
    if 'video' not in request.files or 'audio' not in request.files:
        return jsonify({"error": "Both video and audio files are required"}), 400

    video_file = request.files['video']
    audio_file = request.files['audio']

    # Check if files have valid extensions
    if not allowed_file(video_file.filename) or not allowed_file(audio_file.filename):
        return jsonify({"error": "Invalid file type. Only .mp4 for video and .wav for audio are allowed"}), 400

    # Generate a unique ID for this request to prevent file conflicts
    request_id = str(uuid.uuid4())

    # Save input files with unique names
    video_path = os.path.join(inputPath, f"video_{request_id}.mp4")
    audio_path = os.path.join(inputPath, f"audio_{request_id}.wav")
    lipSyncedOutputPath = os.path.join(outputPath, f"result_{request_id}.mp4")

    video_file.save(video_path)
    audio_file.save(audio_path)

    logging.info(f"Starting inference for request {request_id}")
    future = executor.submit(run_inference, video_path, audio_path, lipSyncedOutputPath)

    try:
        future.result()  # Wait for the task to complete
        logging.info(f"Inference completed for request {request_id}")
        if not os.path.exists(lipSyncedOutputPath):
            logging.error(f"Output file not created for request {request_id}")
            return jsonify({"error": "Output file was not created"}), 500
        return send_file(lipSyncedOutputPath, as_attachment=True)
    except Exception as e:
        logging.error(f"Inference failed for request {request_id}: {str(e)}")
        return jsonify({"error": f"Inference failed: {str(e)}"}), 500

# Start the Flask server
if __name__ == '__main__':
    # Set your ngrok token here (replace with your own token)
    ngrok.set_auth_token("2rrH36aY2OjghIDbMHyH1e9TO8u_674w937bU9yjRn69FhMVX")

    # Expose the Flask app via ngrok
    public_url = ngrok.connect(5000).public_url
    print(f" * Running on {public_url}")

    # Run the Flask app with threading enabled
    app.run(host='0.0.0.0', port=5000, threaded=True)

Cloning into 'Wav2Lip-GFPGAN'...
remote: Enumerating objects: 195, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 195 (delta 18), reused 9 (delta 9), pack-reused 160 (from 1)
Receiving objects: 100% (195/195), 29.94 MiB | 38.51 MiB/s, done.
Resolving deltas: 100% (39/39), done.
/content/Wav2Lip-GFPGAN
--2025-04-07 13:27:54--  https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth
Resolving www.adrianbulat.com (www.adrianbulat.com)... 45.136.29.207
Connecting to www.adrianbulat.com (www.adrianbulat.com)|45.136.29.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89843225 (86M) [application/octet-stream]
Saving to: ‘/content/Wav2Lip-GFPGAN/Wav2Lip-master/face_detection/detection/sfd/s3fd.pth’

/content/Wav2Lip-GF 100%[===================>]  85.68M   200MB/s    in 0.4s    

2025-04-07 13:27:55 (200 MB/s) - ‘/content/Wav2Lip-GFPGAN/Wav2Lip-master/face_detection/detection/sfd/s3fd.p

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [07/Apr/2025 13:43:35] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Apr/2025 13:44:15] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Apr/2025 13:44:52] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Apr/2025 13:45:33] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Apr/2025 13:46:16] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Apr/2025 13:46:33] "POST /process HTTP/1.1" 200 -


In [ ]:
import os
outputPath = basePath+'/outputs'
inputAudioPath = basePath + '/inputs/7b4a3c5d-5043-42a2-97a3-e8cf9dd6c26c.wav'
inputVideoPath = basePath + '/inputs/download.mp4'
lipSyncedOutputPath = basePath + '/outputs/result.mp4'

if not os.path.exists(outputPath):
  os.makedirs(outputPath)

!cd $wav2lipFolderName && python inference.py \
--checkpoint_path checkpoints/wav2lip.pth \
--face {inputVideoPath} \
--audio {inputAudioPath} \
--outfile {lipSyncedOutputPath}

Using cuda for inference.
Traceback (most recent call last):
  File "/content/Wav2Lip-GFPGAN/Wav2Lip-master/inference.py", line 280, in <module>
    main()
  File "/content/Wav2Lip-GFPGAN/Wav2Lip-master/inference.py", line 183, in main
    raise ValueError('--face argument must be a valid path to video/image file')
ValueError: --face argument must be a valid path to video/image file
